In [ ]:
import re
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import TFRobertaModel, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load RoBERTa tokenizer and model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
roberta_model = TFRobertaModel.from_pretrained(model_name)

# Function to clean and preprocess text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Function to preprocess text
def preprocess_text(texts, max_length=128):
    cleaned_texts = [clean_text(text) for text in texts]
    tokens = tokenizer(cleaned_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors="tf")
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']}

# Load dataset
df = pd.read_csv("/kaggle/input/twitter-us-airline/Twitter_US_Airline/Tweets.csv")

# Encode sentiment labels
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['airline_sentiment'])

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['sentiment_encoded'], test_size=0.2, random_state=42)

# Tokenize and preprocess text data
train_inputs = preprocess_text(list(train_texts))
test_inputs = preprocess_text(list(test_texts))

# Convert labels to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Define Ensemble Model: RoBERTa + LSTM + BiLSTM + GRU + MultiHeadAttention
class RobertaEnsembleModel(tf.keras.Model):
    def __init__(self, roberta_model, lstm_units=128, num_classes=3, num_heads=8):
        super(RobertaEnsembleModel, self).__init__()
        self.roberta = roberta_model
        
        # Recurrent layers
        self.lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.gru = tf.keras.layers.GRU(lstm_units, return_sequences=True)
        
        # MultiHeadAttention layers
        self.lstm_attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=lstm_units)
        self.bilstm_attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=lstm_units * 2)
        self.gru_attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=lstm_units)
        
        # Fully connected layers
        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        roberta_outputs = self.roberta(inputs)[0]
        lstm_out = self.lstm(roberta_outputs)
        bilstm_out = self.bilstm(roberta_outputs)
        gru_out = self.gru(roberta_outputs)

        # Apply MultiHeadAttention to each recurrent layer
        lstm_attn = self.lstm_attn(lstm_out, lstm_out)
        bilstm_attn = self.bilstm_attn(bilstm_out, bilstm_out)
        gru_attn = self.gru_attn(gru_out, gru_out)

        # Merge outputs
        merged = self.concat([tf.reduce_mean(lstm_attn, axis=1),
                              tf.reduce_mean(bilstm_attn, axis=1),
                              tf.reduce_mean(gru_attn, axis=1)])
        dropout_out = self.dropout(merged)
        return self.dense(dropout_out)

# Initialize Model
num_classes = len(label_encoder.classes_)
model = RobertaEnsembleModel(roberta_model, lstm_units=128, num_classes=num_classes)

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train Model
model.fit(train_inputs, train_labels, epochs=50, batch_size=32, validation_data=(test_inputs, test_labels))

# Evaluate Model
loss, accuracy = model.evaluate(test_inputs, test_labels)
print(f"Test Accuracy: {accuracy:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/50
366/366 [==============================] - 252s 578ms/step - loss: 0.4942 - accuracy: 0.8051 - val_loss: 0.3904 - val_accuracy: 0.8535
Epoch 2/50
366/366 [==============================] - 207s 565ms/step - loss: 0.3460 - accuracy: 0.8684 - val_loss: 0.3935 - val_accuracy: 0.8538
Epoch 3/50
366/366 [==============================] - 206s 564ms/step - loss: 0.2549 - accuracy: 0.9074 - val_loss: 0.4251 - val_accuracy: 0.8552
Epoch 4/50
366/366 [==============================] - 206s 564ms/step - loss: 0.1843 - accuracy: 0.9360 - val_loss: 0.4474 - val_accuracy: 0.8501
Epoch 5/50
366/366 [==============================] - 207s 565ms/step - loss: 0.1400 - accuracy: 0.9510 - val_loss: 0.5707 - val_accuracy: 0.8473
Epoch 6/50
366/366 [==============================] - 207s 565ms/step - loss: 0.1035 - accuracy: 0.9661 - val_loss: 0.6545 - val_accuracy: 0.8494
Epoch 7/50
366/366 [==============================] - 207s 565ms/step - loss: 0.0850 - accuracy: 0.9716 - val_loss: 0.6345 -

In [ ]:
import re
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import TFRobertaModel, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load RoBERTa tokenizer and model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
roberta_model = TFRobertaModel.from_pretrained(model_name)

# Function to clean and preprocess text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Function to preprocess text
def preprocess_text(texts, max_length=128):
    cleaned_texts = [clean_text(text) for text in texts]
    tokens = tokenizer(cleaned_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors="tf")
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']}

# Load dataset
df = pd.read_csv("/kaggle/input/imdb-50k-movies/IMDb_50K_movies/IMDB Dataset.csv")

# Encode sentiment labels
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'])

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['review'], df['sentiment_encoded'], test_size=0.2, random_state=42)

# Tokenize and preprocess text data
train_inputs = preprocess_text(list(train_texts))
test_inputs = preprocess_text(list(test_texts))

# Convert labels to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Define Ensemble Model: RoBERTa + LSTM + BiLSTM + GRU + MultiHeadAttention
class RobertaEnsembleModel(tf.keras.Model):
    def __init__(self, roberta_model, lstm_units=128, num_classes=3, num_heads=8):
        super(RobertaEnsembleModel, self).__init__()
        self.roberta = roberta_model
        
        # Recurrent layers
        self.lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))
        self.gru = tf.keras.layers.GRU(lstm_units, return_sequences=True)
        
        # MultiHeadAttention layers
        self.lstm_attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=lstm_units)
        self.bilstm_attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=lstm_units * 2)
        self.gru_attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=lstm_units)
        
        # Fully connected layers
        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.dense = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        roberta_outputs = self.roberta(inputs)[0]
        lstm_out = self.lstm(roberta_outputs)
        bilstm_out = self.bilstm(roberta_outputs)
        gru_out = self.gru(roberta_outputs)

        # Apply MultiHeadAttention to each recurrent layer
        lstm_attn = self.lstm_attn(lstm_out, lstm_out)
        bilstm_attn = self.bilstm_attn(bilstm_out, bilstm_out)
        gru_attn = self.gru_attn(gru_out, gru_out)

        # Merge outputs
        merged = self.concat([tf.reduce_mean(lstm_attn, axis=1),
                              tf.reduce_mean(bilstm_attn, axis=1),
                              tf.reduce_mean(gru_attn, axis=1)])
        dropout_out = self.dropout(merged)
        return self.dense(dropout_out)

# Initialize Model
num_classes = len(label_encoder.classes_)
model = RobertaEnsembleModel(roberta_model, lstm_units=128, num_classes=num_classes)

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train Model
model.fit(train_inputs, train_labels, epochs=50, batch_size=32, validation_data=(test_inputs, test_labels))

# Evaluate Model
loss, accuracy = model.evaluate(test_inputs, test_labels)
print(f"Test Accuracy: {accuracy:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/50
1250/1250 [==============================] - 752s 571ms/step - loss: 0.2758 - accuracy: 0.8818 - val_loss: 0.2207 - val_accuracy: 0.9120
Epoch 2/50
1250/1250 [==============================] - 707s 566ms/step - loss: 0.1895 - accuracy: 0.9244 - val_loss: 0.2239 - val_accuracy: 0.9159
Epoch 3/50
1032/1250 [=======================>......] - ETA: 1:53 - loss: 0.1323 - accuracy: 0.9504